In [1]:
%%capture
!pip install koreanize-matplotlib
import koreanize_matplotlib

In [2]:
import pandas as pd
exchange = pd.read_csv('/content/exchange_final.csv')
m2 = pd.read_csv('/content/M2_2000_2024.CSV' , encoding='cp949')

In [3]:
# 1. 날짜 포맷 맞추기
exchange['TIME'] = pd.to_datetime(exchange['TIME']).dt.strftime('%Y-%m')

# 2. m2 인코딩해서 불러오기
m2 = pd.read_csv('/content/M2_2000_2024.CSV', encoding='cp949')  # 또는 euc-kr

# 3. 조인 (왼쪽 기준: exchange 기준으로 붙이기)
df = pd.merge(exchange, m2, on='TIME', how='left')

In [4]:
import random
import numpy as np
import tensorflow as tf
import os

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

- 유의미한 가설로 판정된 변수

In [5]:
# 변수 정의
features = ['경제 심리 지수', '한국 외환 보유액', 'vix', 'WTI 유가', '한국 실업률', '두바이 유가',
                        'Brent 유가', '천연가스 가격', '유연탄 가격', '미국 주가지수',
                        '한국 인플레이션율', '인플레이션 격차', '실질GDP성장률','M2',
                        '금 가격', '구리 가격', '아연 가격', '알루미늄 가격', '니켈 가격',
                        '외국인 투자 금액']

- 조사한 모든 변수

In [ ]:
# 변수 정의
# features = ['경제 심리 지수', '한국 기준 금리', '미국 기준 금리', '한국 외환 보유액',
#             '한국 실업률', '미국 실업률', '한국 주가지수', '미국 주가지수', 'WTI 유가', '두바이 유가',
#             'Brent 유가', '천연가스 가격', '유연탄 가격', '철광석 가격', '구리 가격',
#             '알루미늄 가격', '니켈 가격', '아연 가격', '금 가격', '대두 가격', '외국인 투자 금액',
#             '옥수수 가격', '소맥 가격', '원당 가격', '원면 가격',
#             '한국 경상수지', '한국 인플레이션율', '인플레이션 격차',
#             'M2', '실질GDP성장률', 'vix']

- DJ Twenty → 제일 성능이 좋았던 변수 (이론적 근거 필요)

In [ ]:
# 변수 정의
# features = [
#     '경제 심리 지수', '한국 외환 보유액', '한국 실업률', '미국 주가지수',
#     'WTI 유가', '두바이 유가', 'Brent 유가', '천연가스 가격',
#     '유연탄 가격', '구리 가격', '알루미늄 가격', '니켈 가격',
#     '아연 가격', '금 가격', '한국 상품수지', '한국 인플레이션율',
#     '실질GDP', '실질GDP성장률', 'vix', '외국인 투자 금액'
# ]

In [6]:
target = ['원/달러환율']

# 1. 다음 달 환율을 타깃으로 shift해서 컬럼 생성
df['원/달러환율_t+1'] = df['원/달러환율'].shift(-1)

# 2. target 변수로 지정
target = ['원/달러환율_t+1']

- 결측치 채우기

In [7]:
df_filled = df.interpolate(method='linear')
df_filled = df_filled.fillna(method='ffill').fillna(method='bfill')

<ipython-input-7-76dd81b09891>:1: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_filled = df.interpolate(method='linear')
<ipython-input-7-76dd81b09891>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_filled = df_filled.fillna(method='ffill').fillna(method='bfill')


- 다중공선성 확인 (vif<20)

In [8]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 1. 피처셋 정의
X = df_filled[features].copy()

# 2. VIF 반복 제거 함수 정의
def calculate_vif(df, thresh=20.0):
    variables = df.columns.tolist()
    while True:
        vif = pd.Series(
            [variance_inflation_factor(df[variables].values, i) for i in range(len(variables))],
            index=variables
        )
        max_vif = vif.max()
        if max_vif > thresh:
            drop_feature = vif.idxmax()
            print(f"Removing '{drop_feature}' with VIF={max_vif:.2f}")
            variables.remove(drop_feature)
        else:
            break
    return df[variables]

# 3. 다중공선성 제거된 X 반환
X_vif = calculate_vif(X, thresh=20.0)

Removing 'Brent 유가' with VIF=3259.78
Removing 'WTI 유가' with VIF=504.36
Removing 'M2' with VIF=393.74
Removing '알루미늄 가격' with VIF=304.36
Removing '한국 외환 보유액' with VIF=220.92
Removing '경제 심리 지수' with VIF=152.08
Removing '구리 가격' with VIF=142.76
Removing '미국 주가지수' with VIF=84.02
Removing '아연 가격' with VIF=37.42
Removing '금 가격' with VIF=29.76
Removing '한국 실업률' with VIF=23.58


In [9]:
X_vif.columns # 다중공선성 vif < 20

Index(['vix', '두바이 유가', '천연가스 가격', '유연탄 가격', '한국 인플레이션율', '인플레이션 격차',
       '실질GDP성장률', '니켈 가격', '외국인 투자 금액'],
      dtype='object')

이 9개 피처를 대상으로 다음 세 가지 피처 선택 방법을 차례대로 진행하겠습니다:

- 전진 선택법 (Forward Selection)

- 후진 제거법 (Backward Elimination)

- 단계 선택법 (Stepwise Selection)



In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split

# 타깃 정의
y = df_filled[target].values.flatten()

# 피처셋 정의
X_for_select = X_vif.copy()

# 훈련/검증용 분할 (shuffle=False, 시계열이므로)
X_train_sel, X_val_sel, y_train_sel, y_val_sel = train_test_split(
    X_for_select, y, test_size=0.2, shuffle=False
)

- 전진선택법

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector

# 베이스 모델: 선형회귀 (GRU용 피처선택이므로 복잡도 낮은 모델 사용)
model = LinearRegression()

# 전진 선택법
sfs_forward = SequentialFeatureSelector(
    model,
    n_features_to_select="auto",  # 자동 선택 (교차검증 기반 최적 피처 수)
    direction="forward",
    cv=5,
    scoring='neg_mean_squared_error',  # 최소 MSE 기준
    n_jobs=-1
)

# 학습
sfs_forward.fit(X_train_sel, y_train_sel)

# 선택된 피처
selected_forward_features = X_for_select.columns[sfs_forward.get_support()].tolist()
print("✅ 전진 선택법 결과:")
print(selected_forward_features)

✅ 전진 선택법 결과:
['vix', '천연가스 가격', '인플레이션 격차', '니켈 가격']


- 후진제거법

In [12]:
# 후진 제거법
sfs_backward = SequentialFeatureSelector(
    model,
    n_features_to_select="auto",  # 교차검증 기반 최적 피처 수
    direction="backward",
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# 학습
sfs_backward.fit(X_train_sel, y_train_sel)

# 선택된 피처
selected_backward_features = X_for_select.columns[sfs_backward.get_support()].tolist()
print("✅ 후진 제거법 결과:")
print(selected_backward_features)

✅ 후진 제거법 결과:
['vix', '천연가스 가격', '유연탄 가격', '인플레이션 격차', '니켈 가격']


- 단계선택법

In [13]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression

# 베이스 모델
model = LinearRegression()

# 단계 선택법 (전진 + 후진)
sfs_stepwise = SFS(
    estimator=model,
    k_features='best',          # 최적 피처 수
    forward=True,               # 전진 선택
    floating=True,              # 후진 제거 허용 (SFFS 방식)
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=-1
)

# X_train_sel을 DataFrame으로 넘기기 (columns 유지됨)
sfs_stepwise.fit(X_train_sel, y_train_sel)

# 선택된 피처 이름 추출
selected_stepwise_features = list(sfs_stepwise.k_feature_names_)
print("✅ 단계 선택법 결과:")
print(selected_stepwise_features)

✅ 단계 선택법 결과:
['vix', '두바이 유가', '천연가스 가격', '유연탄 가격', '인플레이션 격차', '실질GDP성장률', '니켈 가격']


## Train Test 분할

In [14]:
from sklearn.model_selection import train_test_split

# 타깃 재정의
y = df_filled[target].values.flatten()

- 전진

In [15]:
X_forward = df_filled[selected_forward_features]
X_train_fwd, X_test_fwd, y_train_fwd, y_test_fwd = train_test_split(
    X_forward, y, test_size=0.2, shuffle=False
)

- 후진

In [16]:
X_backward = df_filled[selected_backward_features]
X_train_bwd, X_test_bwd, y_train_bwd, y_test_bwd = train_test_split(
    X_backward, y, test_size=0.2, shuffle=False
)

- 단계

In [17]:
X_stepwise = df_filled[selected_stepwise_features]
X_train_stp, X_test_stp, y_train_stp, y_test_stp = train_test_split(
    X_stepwise, y, test_size=0.2, shuffle=False
)

## 정규화 (MinMaxScaler)

In [18]:
from sklearn.preprocessing import MinMaxScaler

# 결과 저장할 dict
scalers = {}
X_train_scaled = {}
X_test_scaled = {}

# 함수로 구성
def scale_with_minmax(name, X_train, X_test):
    scaler = MinMaxScaler()
    X_train_scaled[name] = scaler.fit_transform(X_train)
    X_test_scaled[name] = scaler.transform(X_test)
    scalers[name] = scaler

# 전진 선택 피처셋
scale_with_minmax('forward', X_train_fwd, X_test_fwd)

# 후진 제거 피처셋
scale_with_minmax('backward', X_train_bwd, X_test_bwd)

# 단계 선택 피처셋
scale_with_minmax('stepwise', X_train_stp, X_test_stp)

-> 다음과 같이 저장됨

X_train_scaled['forward'], X_test_scaled['forward']

X_train_scaled['backward'], X_test_scaled['backward']

X_train_scaled['stepwise'], X_test_scaled['stepwise']

- 타겟 정규화

In [19]:
# 새로운 MinMaxScaler로 타깃 정규화
target_scaler = MinMaxScaler()
y_train_scaled = target_scaler.fit_transform(y_train_fwd.reshape(-1, 1))
y_test_scaled = target_scaler.transform(y_test_fwd.reshape(-1, 1))

## GRU 모델 학습

In [20]:
import numpy as np
# 전처리: 입력 차원 맞추기
def reshape_for_gru(X):
    return np.reshape(X, (X.shape[0], 1, X.shape[1]))  # (samples, timesteps=1, features)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping

# 입력 데이터 (전진 선택 피처셋 기준)
X_train = reshape_for_gru(X_train_scaled['forward'])
X_test = reshape_for_gru(X_test_scaled['forward'])

# 모델 정의
model = Sequential([
    GRU(32, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# 조기 종료 콜백
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 모델 학습
history = model.fit(
    X_train, y_train_scaled,
    validation_split=0.2,
    epochs=100,
    batch_size=16,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 0.1055 - mae: 0.2810 - val_loss: 0.0839 - val_mae: 0.2807
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0676 - mae: 0.2151 - val_loss: 0.0418 - val_mae: 0.1910
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0393 - mae: 0.1557 - val_loss: 0.0237 - val_mae: 0.1352
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0309 - mae: 0.1310 - val_loss: 0.0195 - val_mae: 0.1197
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0341 - mae: 0.1423 - val_loss: 0.0214 - val_mae: 0.1267
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.0285 - mae: 0.1327 - val_loss: 0.0223 - val_mae: 0.1297
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0253 - mae: 0.1219 - val_loss: 0.0218 - val_mae: 0.1274
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0204 - mae: 0.1068 - val_loss: 0.0212 - val_mae: 0.1243
Epoch 9/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0163 - mae: 

In [24]:
y_pred_scaled = model.predict(X_test)
y_pred = target_scaler.inverse_transform(y_pred_scaled)  # 역변환

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 평가
rmse = np.sqrt(mean_squared_error(y_test_fwd, y_pred))
mae = mean_absolute_error(y_test_fwd, y_pred)
r2 = r2_score(y_test_fwd, y_pred)

print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R²: {r2:.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
RMSE: 199.5972
MAE: 174.9192
R²: -3.4243


In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

set_seed(42)

# reshape for GRU
def reshape_for_gru(X):
    return np.reshape(X, (X.shape[0], 1, X.shape[1]))

# GRU 학습 및 평가 함수
def train_and_evaluate_gru(name, X_train_raw, X_test_raw, y_train, y_test):
    print(f"\n✅ [{name.upper()}] GRU 모델 학습 시작")

    # 타깃 정규화
    y_scaler = MinMaxScaler()
    y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1))
    y_test_scaled = y_scaler.transform(y_test.reshape(-1, 1))

    # 입력 reshape
    X_train = reshape_for_gru(X_train_raw)
    X_test = reshape_for_gru(X_test_raw)

    # 모델 정의
    model = Sequential([
        GRU(32, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])

    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # 학습
    model.fit(
        X_train, y_train_scaled,
        validation_split=0.2,
        epochs=100,
        batch_size=16,
        callbacks=[early_stop],
        verbose=0  # 로그 생략
    )

    # 예측 → 역변환
    y_pred_scaled = model.predict(X_test)
    y_pred = y_scaler.inverse_transform(y_pred_scaled)

    # 성능 평가
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"RMSE: {rmse:.4f} | MAE: {mae:.4f} | R²: {r2:.4f}")
    return rmse, mae, r2

In [26]:
# 전진 선택
train_and_evaluate_gru('forward', X_train_scaled['forward'], X_test_scaled['forward'], y_train_fwd, y_test_fwd)

# 후진 제거
train_and_evaluate_gru('backward', X_train_scaled['backward'], X_test_scaled['backward'], y_train_bwd, y_test_bwd)

# 단계 선택
train_and_evaluate_gru('stepwise', X_train_scaled['stepwise'], X_test_scaled['stepwise'], y_train_stp, y_test_stp)


✅ [FORWARD] GRU 모델 학습 시작


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
RMSE: 209.5259 | MAE: 184.0652 | R²: -3.8754

✅ [BACKWARD] GRU 모델 학습 시작


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


RMSE: 133.2943 | MAE: 101.0887 | R²: -0.9731

✅ [STEPWISE] GRU 모델 학습 시작
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
RMSE: 149.7993 | MAE: 127.3153 | R²: -1.4920


(np.float64(149.7992613727133), 127.31526472981768, -1.4920437813350262)